<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Required Importing**

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

###**Uploading Dataset**

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Content_Project/data.csv')

In [5]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


##**Preprocessing Dataset**

In [6]:
#Checking if NULL rows are present
df.isna().sum()

_id                0
title              0
datePublished      0
description        0
article           21
keywords         919
dtype: int64

In [7]:
#Removing the row with null values
df.dropna(inplace=True)

In [8]:
#Making new dataset 
df1=df[:500]

In [9]:
df1.to_csv('data1.csv',index=False)

In [10]:
data=pd.read_csv('data1.csv')

In [11]:
data.shape

(500, 6)

In [12]:
#Removing duplicates rows
data.drop_duplicates(inplace=True)

####**Now check the language of texts**

In [13]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import langdetect

In [15]:
def is_english(text):
    try:
        lang = langdetect.detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

In [16]:
data['is_english'] = data['article'].apply(lambda x: is_english(x))

In [17]:
data.head()

,_id,title,datePublished,description,article,keywords,is_english
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream...",True
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct...",True
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ...",True
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par...",True
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube",True


In [18]:
data['is_english'].value_counts()

True    500
Name: is_english, dtype: int64

In [19]:
data= data[data['is_english']==True]

###**Text Length Count**

In [20]:
from collections import Counter

In [21]:
data['text_length'] = data['article'].apply(len)

In [22]:
data.sort_values(by='text_length')

,_id,title,datePublished,description,article,keywords,is_english,text_length
463,{'$oid': '60083b28b5425d67582e2844'},"As Uber and Lyft continue to melt, the 2019 un...",2020-03-18 08:15:21,You’d be excused for feeling that mid-2019 was...,You’d be excused for feeling that mid-2019 wa...,"2020, crowdstrike, Lyft, SaaS, slack, stock ma...",True,739
58,{'$oid': '600837ecb5425d67582e0f76'},Survey: Tell us what you think of Extra Crunch...,2020-08-07 10:26:33,Since launching our membership product Extra C...,Since launching our membership product Extra ...,Extra Crunch,True,743
425,{'$oid': '60083adfb5425d67582e25ae'},Live from Apple’s virtual WWDC 2020,2020-06-22 09:20:49,This WWDC is going to be one to remember — if ...,This WWDC is going to be one to remember — if ...,"Apple, iOS, macos, WWDC 2020",True,773
242,{'$oid': '6008394eb5425d67582e1a2c'},Will China’s coronavirus-related trends shape ...,2020-04-21 15:40:40,There may be dark horses waiting to break out ...,"For the past month, VC investment pace seems ...","alibaba group, artificial intelligence, China,...",True,807
364,{'$oid': '60083ad2b5425d67582e2524'},Live from Apple’s virtual 2020 iPhone event – ...,2020-10-13 10:02:57,Apple’s big iPhone event is finally here — vir...,Apple’s big iPhone event is finally here — vir...,"Apple, apple inc, apple iphone event 2020, com...",True,857
...,...,...,...,...,...,...,...,...
331,{'$oid': '600839cab5425d67582e1e23'},Cookie consent tools are being used to undermi...,2020-01-10 10:08:53,Most cookie consent pop-ups served to internet...,Most cookie consent pop-ups served to internet...,"Alexa, consent, cookie pop-ups, Cookiebot, dat...",True,14797
55,{'$oid': '600837c1b5425d67582e0e57'},A look back at 10 years of CES,2020-01-05 15:35:41,"So, here we are on the eve of CES 2020 — the s...","So, here we are on the eve of CES 2020 — the ...","CES, CES 2020",True,15641
160,{'$oid': '600838e5b5425d67582e1700'},"Hands on with Telepath, the social network tak...",2020-10-11 08:09:07,There’s no doubt that modern social networks h...,There’s no doubt that modern social networks h...,"america, anti, biden, ceo, computing, Facebook...",True,17333
103,{'$oid': '600838a9b5425d67582e150d'},Europe sets out plan to boost data reuse and r...,2020-02-19 06:20:10,European Union lawmakers have set out a first ...,European Union lawmakers have set out a first ...,"artificial intelligence, data economy, data pr...",True,18357


####**Now consider articles with length between (300 and 30000)**

In [23]:
data= data[(data['text_length'] >= 300) & (data['text_length'] <= 20000)]

In [24]:
data.shape

(499, 8)

##**Split the dataset into train and evaluate**

In [25]:
train_data, eval_data = train_test_split(data, test_size=0.2)

In [26]:
train_data.shape

(399, 8)

In [27]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')

In [28]:
# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [29]:
# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


##**Load the model and tokenizer**

In [30]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

##**Prepare your dataset by creating a TextDataset object**

In [31]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [32]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


##**Set up the trainer with the appropriate training arguments**

In [33]:
#Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_total_limit=2,
)

#Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
   # callbacks=[checkpoint_callback]
)



##**Train the model**

In [34]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.454652
2,No log,3.406475
3,No log,3.395971
4,3.453400,3.392081
5,3.453400,3.394321


TrainOutput(global_step=825, training_loss=3.336743016098485, metrics={'train_runtime': 475.9475, 'train_samples_per_second': 27.608, 'train_steps_per_second': 1.733, 'total_flos': 858344325120000.0, 'train_loss': 3.336743016098485, 'epoch': 5.0})

In [35]:
trainer.evaluate()

{'eval_loss': 3.3943207263946533,
 'eval_runtime': 7.0095,
 'eval_samples_per_second': 96.583,
 'eval_steps_per_second': 12.126,
 'epoch': 5.0}

###**Now code to rehit the promt to get required length article**

In [43]:

# Set the maximum length of each segment
max_segment_length = 300

# Set the initial prompt
prompt = "Artificial intelligence"

# Initialize the generated text
generated_text = prompt

# Generate segments of text and stitch them together
while len(generated_text) < 500:
    # Encode the prompt as input IDs
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate the next segment of text
    output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             temperature=0.7,
                             top_k=50)
    
    # Decode the output to text
    segment = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the segment to the generated text
    generated_text += segment

    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

# Print the generated text
print(generated_text)
final_text=generated_text


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: ignored

In [37]:
print(len(final_text))

1024


In [38]:
type(final_text)

str

##**Use text summarization to improve article**

In [39]:
# Install required libraries
#!pip install transformers

# Import required libraries
from transformers import pipeline

# Load pre-trained summarization model
#summarizer = pipeline("summarization")
# Load pre-trained summarization model (using BART-Large-CNN)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


# Define input text to summarize
input_text = "The United States is a country located in North America. It is the third largest country in the world by total area and the third most populous country in the world. The United States has a diverse geography and climate, with mountains, forests, deserts, and coastal areas. Its economy is the largest in the world, driven by industries such as technology, finance, and healthcare."

# Generate summary for input text
summary = summarizer(final_text, max_length=1000, min_length=30, do_sample=False)

# Print generated summary
print(summary[0]['summary_text'])


Your max_length is set to 1000, but you input_length is only 204. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)


New tools are being made available to developers through the “Digital Machines’ platform, which is open source and accessible to the public. Developers can “use the Digital Machines” platform to connect with a number of existing and emerging AI capabilities.


##**Below code is for save the model**

In [40]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [41]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

In [ ]:
loaded_model_joblib

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')